In [18]:
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
import pandas as pd
import util as util
import data_pipeline as data_pipeline
import preprocessing as preprocessing

In [19]:
config_data = util.load_config()
ohe_ContractRenewal = util.pickle_load(config_data["ohe_ContractRenewal_path"])
ohe_DataPlan = util.pickle_load(config_data["ohe_DataPlan_path"])
le_encoder = util.pickle_load(config_data["le_encoder_path"])
model_data = util.pickle_load(config_data["production_model_path"])

C:\Users\Axel\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator OneHotEncoder from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Axel\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\Axel\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.1.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own r

In [20]:
class api_data(BaseModel):
    AccountWeeks : int   
    ContractRenewal : str
    DataPlan : str
    DataUsage : float
    CustServCalls : int
    DayMins : float
    DayCalls : int
    MonthlyCharge : float
    OverageFee : float
    RoamMins : float

In [21]:
app = FastAPI()

In [22]:
@app.get("/")
def home():
    return "Hello, FastAPI up!"

In [27]:
def predict(data: api_data):    
    # Convert data api to dataframe
    data = pd.DataFrame(data).set_index(0).T.reset_index(drop = True)

    # Convert dtype
    data = pd.concat(
        [
            data[config_data["predictors"][0]],
            data[config_data["predictors"][1:]].astype(int)
        ],
        axis = 1
    )

    # Check range data
    try:
        data_pipeline.check_data(data, config_data, True)
    except AssertionError as ae:
        return {"res": [], "error_msg": str(ae)}
    
    # Encoding ContractRenewal
    data = preprocessing.ohe_transform_ContractRenewal(data, "ContractRenewal", ohe_ContractRenewal)
    
    #Cek lagi
    # Encoding DataPlan
    data = preprocessing.ohe_transform_DataPlan(data, "DataPlan", ohe_DataPlan)

    # Predict data
    y_pred = model_data["model_data"]["model_object"].predict(data)

    # Inverse tranform
    y_pred = list(le_encoder.inverse_transform(y_pred))[0] 

    return {"res" : y_pred, "error_msg": ""}


In [30]:
uvicorn.run("api:app", host = "0.0.0.0", port = 8080)

RuntimeError: asyncio.run() cannot be called from a running event loop

In [29]:
data = pd.DataFrame(data).set_index(0).T.reset_index(drop = True)

NameError: name 'data' is not defined